# Analyze air traffic ✈

|Nom | Prénom|
|---|---|
| Malova | Arina |
| Tersou | Victor |


On commence par importer tous les packages dont nous allons avoir besoin

In [1]:
!pip install --upgrade pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install mlforecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import pandas as pd
from pandas_profiling import ProfileReport
import pandas as pd
from prophet import Prophet
import datetime
import plotly
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.subplots import make_subplots

On charge les données

In [4]:
traffic_df = pd.read_parquet('traffic_10lines.parquet')

Le dataframe ci-dessous nous permet de savoir quelles routes existent. 

In [21]:
data = [
    ('LGW', 'BCN'),
    ('LGW', 'AMS'),
    ('LIS', 'ORY'),
    ('LIS', 'OPO'),
    ('SSA', 'GRU'),
    ('NTE', 'FUE'),
    ('LYS', 'PIS'),
    ('PNH', 'NGB'),
    ('POP', 'JFK'),
    ('SCL', 'LHR')
]


df = pd.DataFrame(data, columns=['Home airport', 'Paired airport'])

df

,Home airport,Paired airport
0,LGW,BCN
1,LGW,AMS
2,LIS,ORY
3,LIS,OPO
4,SSA,GRU
5,NTE,FUE
6,LYS,PIS
7,PNH,NGB
8,POP,JFK
9,SCL,LHR


On génère deux fonctions :

- Une fonction `generate_route_df` qui a pour objectif d'extraire une route du dataframe *traffic_df* pour une aéroport de départ et celui d'arrivée. Cette fonction prend 3 arguments : le dataframe, le code IATA pour l'aéroport de départ et le code IATA pour l'aéroport d'arrivée. La fonction retourne un dataframe du trafic entre les deux aéroports par date. 

- Une fonction `run_prophet_forecast` qui a pour objectif de prédire le trafic pour une route donnée et une période donnée. Cette fonction prend 4 arguments : le dataframe utilisée, l'aéroport de départ, l'aéroport d'arrivée et le nombre de périodes à prédire dans le futur. La fonction retourne un dataframe de prédictions. 

Pour tester nos fonctions nous utiliserons la route suivante : LGW - BCN.

In [16]:
def generate_route_df(traffic_df, homeAirport, pairedAirport):
    _df = (traffic_df
           .query('home_airport == "{home}" and paired_airport == "{paired}"'.format(home=homeAirport, paired=pairedAirport))
           .groupby(['home_airport', 'paired_airport', 'date'])
           .agg(pax_total=('pax', 'sum'))
           .reset_index()
           )
    return _df

def run_prophet_forecast(traffic_df, homeAirport, pairedAirport, forecast_periods):
    route_df = generate_route_df(traffic_df, homeAirport, pairedAirport).rename(columns={'date': 'ds', 'pax_total': 'y'})

    baseline_model = Prophet()
    baseline_model.fit(route_df)

    future_df = baseline_model.make_future_dataframe(periods=forecast_periods)
    forecast_df = baseline_model.predict(future_df)

    return forecast_df

forecast_result = run_prophet_forecast(traffic_df, "LGW", "BCN", 15)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyndw54f6/dyd0ejdc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyndw54f6/m9vb7dir.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=35166', 'data', 'file=/tmp/tmpyndw54f6/dyd0ejdc.json', 'init=/tmp/tmpyndw54f6/m9vb7dir.json', 'output', 'file=/tmp/tmpyndw54f6/prophet_modelebf7xd2g/prophet_model-20230601164235.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:42:35 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:42:36 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


On génère une fonction `draw_ts_multiple` qui a pour objectif de plot une série temporelle pour une route donnée. Cette fonction prend 8 arguments : le dataframe utilisé, l'axe des abscisses, l'axe des ordonnées, le nom des prédictions représentées par une ligne pointillée, la date, l'utilisation d'un axis des ordonnées secondaire, si l'on souhaite une zone pour le covid et si l'on souhaite afficher le graph. La fonction retourne une graph. 

In [17]:
def draw_ts_multiple(df, v1, v2=None, prediction=None, date='date', secondary_y=True, covid_zone=False, display=True):
    if isinstance(v1, str):
        variables = [(v1, 'Actual')]
    else:
        variables = [(v, 'V1.{}'.format(i)) for i, v in enumerate(v1)]
    title = '<br>'.join([n + ': ' + v for v, n in variables]) + ('<br>V2: ' + v2) if v2 else '<br>'.join(
        [v + ': ' + n for v, n in variables])
    layout = dict(
        title=title,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type='date'
        )
    )
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(layout)
    for v, name in variables:
        fig.add_trace(go.Scatter(x=df[date], y=df[v], name=name), secondary_y=False)
    if v2:
        fig.add_trace(go.Scatter(x=df[date], y=df[v2], name='V2'), secondary_y=secondary_y)
        fig['layout']['yaxis2']['showgrid'] = False
        fig.update_yaxes(rangemode='tozero')
        fig.update_layout(margin=dict(t=125 + 30 * (len(variables) - 1)))
    if prediction:
        fig.add_trace(go.Scatter(x=forecast_result["ds"], y=forecast_result["yhat"], name='Prediction', line={'dash': 'dot'}), secondary_y=False)

    if covid_zone:
        fig.add_vrect(
            x0=pd.Timestamp("2020-03-01"), x1=pd.Timestamp("2022-01-01"),
            fillcolor="Gray", opacity=0.5,
            layer="below", line_width=0,
        )
    if display:
      pyoff.iplot(fig)
    return fig

In [18]:
draw_ts_multiple(
    (traffic_df
     .query('home_airport == "LGW" and paired_airport == "BCN"')
     .groupby(['home_airport', 'paired_airport', 'date'])
     .agg(pax_total=('pax', 'sum'))
     .reset_index()
    ),
    'pax_total',
    covid_zone=True,
    prediction=True
)